# Day-3: Render Infographic
Reads Day-2 outputs (`data/entropy_sieve_ci.json`, optional `capsules/SIEVE_DAY2.json`),
computes summary metrics, validates against `schema/infographic.schema.json`,
and writes `docs/day3_infographic.png` + `docs/day3_infographic.json`.

In [ ]:
import json, os, datetime, pathlib, math
from jsonschema import validate
import matplotlib.pyplot as plt

ROOT = pathlib.Path('.')
SIEVE_META = ROOT / 'data' / 'entropy_sieve_ci.json'
DAY2_CAPSULE = ROOT / 'capsules' / 'SIEVE_DAY2.json'
SCHEMA_PATH = ROOT / 'schema' / 'infographic.schema.json'
OUT_PNG = ROOT / 'docs' / 'day3_infographic.png'
OUT_JSON = ROOT / 'docs' / 'day3_infographic.json'

OUT_PNG.parent.mkdir(parents=True, exist_ok=True)

def load_json(p):
    with open(p, 'r', encoding='utf-8') as f:
        return json.load(f)

meta = load_json(SIEVE_META) if SIEVE_META.exists() else {}
cap  = load_json(DAY2_CAPSULE) if DAY2_CAPSULE.exists() else {}
schema = load_json(SCHEMA_PATH)

# Fallbacks if fields are absent
samples = int(meta.get('samples', meta.get('N_TRACES', 0)) or 0)
trace_len = int(meta.get('trace_len', meta.get('TRACE_LEN', 0)) or 0)
np_hits = int(meta.get('np_wall_hits', 0))
np_threshold = float(meta.get('np_threshold', 0.09))
p_threshold  = float(meta.get('p_threshold', 0.045))

np_pct = (np_hits / samples * 100.0) if samples > 0 else 0.0

summary = {
  'id': 'INFO_DAY3',
  'version': '1.0.0',
  'title': 'Day-2 Entropy Sieve — NP-wall incidence',
  'subtitle': 'Irreversible spikes above ΔΦ threshold',
  'created_at': datetime.datetime.utcnow().isoformat() + 'Z',
  'sources': {
    'sieve_meta': str(SIEVE_META),
    'capsule': str(DAY2_CAPSULE) if DAY2_CAPSULE.exists() else '',
    'commit': os.environ.get('GITHUB_SHA', ''),
    'run_id': os.environ.get('GITHUB_RUN_ID', '')
  },
  'metrics': {
    'np_wall_pct': round(np_pct, 3),
    'np_threshold': np_threshold,
    'p_threshold': p_threshold,
    'samples': samples,
    'trace_len': trace_len,
    'counts': {
      'np_wall': np_hits,
      'p_like': int(samples - np_hits) if samples else 0,
      'unknown': 0
    }
  },
  'visuals': {
    'output_png': str(OUT_PNG),
    'output_svg': str(OUT_PNG.with_suffix('.svg'))
    'palette': ['#111827', '#4B5563', '#10B981'],
    'theme': 'dark'
  },
  'notes': [
    'Auto-generated from Day-2 CI outputs',
    'np_wall_pct = np_wall_hits / samples * 100'
  ]
}

validate(instance=summary, schema=schema)
print('Metadata validated ✅')

# --- Plot ---
fig, ax = plt.subplots(figsize=(6, 4), dpi=160)
ax.bar(['NP-wall %'], [summary['metrics']['np_wall_pct']])
ax.set_ylim(0, 100)
ax.set_ylabel('Percent of traces')
ax.set_title('NP-wall incidence (ΔΦ > {:.3f}) — samples: {}'.format(np_threshold, samples))
for p in ax.patches:
    h = p.get_height()
    ax.annotate(f"{h:.2f}%", (p.get_x()+p.get_width()/2, h), ha='center', va='bottom')
fig.tight_layout()
fig.savefig(OUT_PNG)
fig.savefig(summary['visuals']['output_svg'])
plt.close(fig)
print('Saved', OUT_PNG)

with open(OUT_JSON, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2)
print('Wrote', OUT_JSON)

from IPython.display import Image, display
display(Image(filename=str(OUT_PNG)))